In [1]:
!pip install -q tensorflow-hub
!pip install -q tfds-nightly

ERROR: tensorflow-metadata 0.25.0 has requirement absl-py<0.11,>=0.9, but you'll have absl-py 0.11.0 which is incompatible.


In [2]:
import tensorflow_hub as hub
import tensorflow_datasets as tfds
import tensorflow as tf
import numpy as np

In [3]:
print("버전: ", tf.__version__)
print("즉시 실행 모드: ", tf.executing_eagerly())
print("허브 버전: ", hub.__version__)
print("GPU", "사용 가능" if tf.config.experimental.list_physical_devices("GPU") else "사용 불가능")

버전:  2.3.1
즉시 실행 모드:  True
허브 버전:  0.10.0
GPU 사용 불가능


#### < 데이터셋 다운로드 >

In [5]:
train_data, validation_data, test_data = tfds.load(
                                         name="imdb_reviews",
                                         split=("train[:60%]", "train[60%:]","test"),
                                         as_supervised=True)

Shuffling and writing examples to C:\Users\user\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteY2NBIA\imdb_reviews-train.tfrecord


Shuffling and writing examples to C:\Users\user\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteY2NBIA\imdb_reviews-test.tfrecord


Shuffling and writing examples to C:\Users\user\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteY2NBIA\imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to C:\Users\user\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.


#### < 데이터 탐색 >

In [6]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))

# 10개의 샘플 출력
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [7]:
# 처음 10개의 레이블 출력
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0], dtype=int64)>

#### < 모델 구성 >

In [8]:
# 텐서플로 허브에 있는 사전 훈련된 텍스트 임베딩 모델인 
# google/tf2-preview/gnews-swivel-20dim/1을 사용

In [9]:
# 테스트해 볼 수 있는 사전 훈련된 모델이 세 개 더 있음
# google/tf2-preview/nnlm-en-dim50/1 (더 큰 모델) : 차원 크기는 50이고 어휘 사전의 크기는 1백만 개 이하
# google/tf2-preview/nnlm-en-dim128/1 (훨씬 더 큰 모델) : 차원 크기는 128이고 어휘 사전의 크기는 1백만 개 이하

In [10]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"

hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)

# 몇 개의 샘플을 입력하여 테스트 진행
hub_layer(train_examples_batch[:3])

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [11]:
# 입력 텍스트의 길이에 상관없이 임베딩의 출력 크기는 
# (num_examples, embedding_dimension)가 됨

In [12]:
# 전체 모델 생성 
model = tf.keras.Sequential()

# 순서대로 층을 쌓아 분류기를 만듦
model.add(hub_layer) # 첫 번째 층은 텐서플로 허브 층
model.add(tf.keras.layers.Dense(16, activation='relu')) # 16개의 은닉 유닛을 가진 완전 연결 층(Dense)
model.add(tf.keras.layers.Dense(1)) # 하나의 출력 노드를 가진 완전 연결 층

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 20)                400020    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 400,373
Trainable params: 400,373
Non-trainable params: 0
_________________________________________________________________


In [14]:
# 옵티마이저와 손실 함수를 설정
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

#### < 모델 훈련 >

In [15]:
# 이 모델을 512개의 샘플로 이루어진 미니배치(mini-batch)에서 
# 20번의 에포크(epoch) 동안 훈련을 수행
# (x_train과 y_train 텐서에 있는 모든 샘플에 대해 20번 반복)

# 훈련하는 동안 10,000개의 검증 세트에서 모델의 손실과 정확도를 모니터링

In [16]:
history = model.fit(train_data.shuffle(10000).batch(512), epochs=20,
                    validation_data=validation_data.batch(512), verbose=1)

Epoch 1/20
30/30 [==============================] - 3s 86ms/step - loss: 0.6909 - accuracy: 0.5497 - val_loss: 0.6191 - val_accuracy: 0.6132
Epoch 2/20
30/30 [==============================] - 2s 66ms/step - loss: 0.5938 - accuracy: 0.6489 - val_loss: 0.5681 - val_accuracy: 0.6747
Epoch 3/20
30/30 [==============================] - 2s 66ms/step - loss: 0.5447 - accuracy: 0.6963 - val_loss: 0.5280 - val_accuracy: 0.7221
Epoch 4/20
30/30 [==============================] - 2s 65ms/step - loss: 0.4996 - accuracy: 0.7376 - val_loss: 0.4890 - val_accuracy: 0.7505
Epoch 5/20
30/30 [==============================] - 2s 67ms/step - loss: 0.4563 - accuracy: 0.7758 - val_loss: 0.4549 - val_accuracy: 0.7688
Epoch 6/20
30/30 [==============================] - 2s 66ms/step - loss: 0.4166 - accuracy: 0.8019 - val_loss: 0.4254 - val_accuracy: 0.7908
Epoch 7/20
30/30 [==============================] - 2s 70ms/step - loss: 0.3819 - accuracy: 0.8245 - val_loss: 0.4005 - val_accuracy: 0.8085
Epoch 8/20
30

#### < 모델 평가 >

In [17]:
results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

49/49 - 2s - loss: 0.3256 - accuracy: 0.8610
loss: 0.326
accuracy: 0.861


In [ ]:
# 손실과 오차, 두 개의 값이 반환됩